## Notebook that trains a regular GAN for generating just galaxies
Implementation adapted from: https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
and
https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/dcgan/dcgan.py

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.utils import save_image
import torchvision
from PIL import Image
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
import torchvision.utils as vutils
import matplotlib.animation as animation
from IPython.display import HTML

# set up accordingly
data_dir = "../../../data" # directory with data files
labeled_image_dir = "labeled_patches"           # folder within data directory with labeled images (0, 1)
scored_image_dir = "scored_patches"             # folder within data directory with scored images
labeled_info = "labeled_patches_t5.csv"            # information about labeled galaxy patches
scored_info = "scored_patches_t5.csv"
device = torch.device("cpu")        # cuda or cpu
collab = False                        # google collab flag

In [2]:
# google collab
# mount drive, copy over data as zip and unzip it
if collab:
  from google.colab import drive
  drive.mount('/content/drive')
  collab_dir = "content"

  # zip_path = os.path.join(data_dir, labeled_image_dir + '.tar.gz')
  # !cp '{zip_path}' .
  # !tar -xzf '{labeled_image_dir}'.tar.gz

  # zip_path = os.path.join(data_dir, scored_image_dir + '.tar.gz')
  # !cp '{zip_path}' .
  # !tar -xzf '{scored_image_dir}'.tar.gz

  zip_path = os.path.join(data_dir, labeled_image_dir + '.zip')
  !cp '{zip_path}' .
  !unzip -q '{labeled_image_dir}'.zip
  !mv labeled_patches '{labeled_image_dir}'

  zip_path = os.path.join(data_dir, scored_image_dir + '.zip')
  !cp '{zip_path}' .
  !unzip -q '{scored_image_dir}'.zip
  !mv scored_patches '{scored_image_dir}'
else:
  labeled_image_dir = os.path.join(data_dir,labeled_image_dir)
  scored_image_dir = os.path.join(data_dir,scored_image_dir)

In [3]:
class GalaxyDataset(torch.utils.data.Dataset):
    """
    Galaxy dataset class
    Builds a dataset from the labeled and scored images. 
    Requires a threshold score for scored images. 
    Images with a score below the threshold are not used.
    """

    def __init__(self, labeled_csv_file, labeled_image_dir, scored_csv_file=None, scored_image_dir=None, transform=None, size=(32, 32), scored_threshold=3):
        self.image_ids = pd.read_csv(labeled_csv_file, index_col="patch_id")
        if scored_csv_file is not None and scored_image_dir is not None:
          self.image_ids = self.image_ids.append(pd.read_csv(scored_csv_file, index_col="patch_id"))
        self.size = size
        self.samples = []
        self.labeled_image_dir = labeled_image_dir
        self.scored_image_dir = scored_image_dir
        self.transform = transform
        self.scored_threshold = scored_threshold
        self.load_dataset()

    def __len__(self):
        return len(self.samples)

    def load_dataset(self):
      print("Loading Dataset...")
      for id, data in self.image_ids.iterrows():
        score = data.score
        if score == -1:
          img_name = os.path.join(self.labeled_image_dir,
                                  str(id))
        else:
          img_name = os.path.join(self.scored_image_dir,
                                  str(id))
        # labeled images have score of -1
        # have already been filtered so only real images are left
        if score == -1 or score >= self.scored_threshold:
          self.samples.append(Image.open(img_name).convert('L'))
        
      print("Dataset Loaded")

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.samples[idx]
        if self.transform:
            image = self.transform(image)

        return image

In [6]:
batch_size = 64
size = (32, 32)
latent_dim = 100
train_transformation = torchvision.transforms.Compose([
                            torchvision.transforms.RandomAffine(0, translate=(0.2, 0.2)),
                            torchvision.transforms.ToTensor(),
                            torchvision.transforms.Normalize((0.5,), (0.5,)) ## second normalization for tanh
])
val_transformation = torchvision.transforms.Compose([
                            torchvision.transforms.ToTensor(),
                            torchvision.transforms.Normalize((0.5,), (0.5,)) ## second normalization for tanh
])


train_dataset = GalaxyDataset(os.path.join(data_dir, labeled_info),
                              labeled_image_dir,
                              scored_csv_file=os.path.join(data_dir, scored_info),
                              scored_image_dir=scored_image_dir,
                              transform=train_transformation,
                              size=size,
                              scored_threshold=3)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=1, pin_memory=True)
print("{} images loaded".format(len(train_dataset)))

Loading Dataset...
Dataset Loaded
34576 images loaded


In [5]:
gen_base_filters = 64
disc_base_filters = 64

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( latent_dim, gen_base_filters * 4, 4, 1, 0, bias=False),
            nn.ReLU(True),
            nn.BatchNorm2d(gen_base_filters * 4),
            # state size. (gen_base_filters*4) x 4 x 4
            nn.ConvTranspose2d(gen_base_filters * 4, gen_base_filters * 2, 4, 2, 1, bias=False),
            nn.ReLU(True),
            nn.BatchNorm2d(gen_base_filters * 2),
            # state size. (gen_base_filters*2) x 8 x 8
            nn.ConvTranspose2d( gen_base_filters * 2, gen_base_filters, 4, 2, 1, bias=False),
            nn.ReLU(True),
            nn.BatchNorm2d(gen_base_filters),
            # state size. (gen_base_filters) x 16 x 16
            nn.ConvTranspose2d( gen_base_filters, 1, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. 1 x 32 x 32
        )

    def forward(self, input):
        return self.main(input)

In [7]:
gen = Generator().to(device)
gen.apply(weights_init)
print(gen)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): ReLU(inplace=True)
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): ReLU(inplace=True)
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): Tanh()
  )
)


In [8]:
class Discriminator(nn.Module):
    def __init__(self, noise=False):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # input is 1 x 32 x 32
            nn.Conv2d(1, disc_base_filters, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (disc_base_filters) x 16 x 16
            nn.Conv2d(disc_base_filters, disc_base_filters * 2, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm2d(disc_base_filters * 2),
            # state size. (disc_base_filters*2) x 8 x 8
            nn.Conv2d(disc_base_filters * 2, disc_base_filters * 4, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm2d(disc_base_filters * 4),
            # state size. (disc_base_filters*4) x 4 x 4
            nn.Conv2d(disc_base_filters * 4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
        self.noise=noise

    def forward(self, input):
        return self.main(input)

In [9]:
disc = Discriminator().to(device)
disc.apply(weights_init)
print(disc)

Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): LeakyReLU(negative_slope=0.2, inplace=True)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (9): Sigmoid()
  )
)


Set up training parameters

In [10]:
G_losses = []
D_losses = []
iters = 0
img_list = []

# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 0.9
fake_label = 0
gen_lr = 1e-4
disc_lr = 2e-4
epochs = 1000
# generator takes this many steps for every 1 of the discriminator
gen_steps = 1
# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(disc.parameters(), lr=disc_lr, betas=(0.5, 0.999))
optimizerG = optim.Adam(gen.parameters(), lr=gen_lr, betas=(0.5, 0.999))

Function that takes one GAN step

In [11]:
def gan_step(batch_num, batch, disc, gen):
  ############################
  # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
  ###########################
  ## Train with all-real batch
  disc.zero_grad()
  # Format batch
  real_cpu = data.to(device)
  b_size = real_cpu.size(0)
  label = torch.full((b_size,), real_label, device=device)
  # Forward pass real batch through D
  output = disc(real_cpu).view(-1)
  # Calculate loss on all-real batch
  errD_real = criterion(output, label)
  # Calculate gradients for D in backward pass
  errD_real.backward()
  D_x = output.mean().item()

  ## Train with all-fake batch
  # Generate batch of latent vectors
  noise = torch.randn(b_size, latent_dim, 1, 1, device=device)
  # Generate fake image batch with G
  fake = gen(noise)
  label.fill_(fake_label)
  # Classify all fake batch with D
  output = disc(fake.detach()).view(-1)
  # Calculate D's loss on the all-fake batch
  errD_fake = criterion(output, label)
  # Calculate the gradients for this batch
  errD_fake.backward()
  D_G_z1 = output.mean().item()
  # Add the gradients from the all-real and all-fake batches
  errD = errD_real + errD_fake
  

  ############################
  # (2) Update G network: maximize log(D(G(z)))
  # try training discriminator more than generator
  ###########################
  for _ in range (gen_steps):
    gen.zero_grad()
    label.fill_(real_label)  # fake labels are real for generator cost
    # Since we just updated D, perform another forward pass of all-fake batch through D
    output = disc(fake).view(-1)
    # Calculate G's loss based on this output
    errG = criterion(output, label)
    # Calculate gradients for G
    errG.backward()
    D_G_z2 = output.mean().item()

  # Output training stats
  if i % 5 == 0:
      print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
            % (epoch, epochs, i, len(train_loader),
                errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
  return errD, errG

Begin training and periodically save generator output

In [12]:
results_dir = "results_galaxy_gan_t3"
if not os.path.exists(os.path.join(results_dir)):
  os.mkdir(os.path.join(results_dir))
  os.mkdir(os.path.join(results_dir, "normalized"))
  os.mkdir(os.path.join(results_dir, "unnormalized"))
for epoch in range(1, epochs + 1):
    # For each batch in the dataloader
    for i, data in enumerate(train_loader, 0):

        errD, errG = gan_step(i, data, disc, gen)
        
        # Update G
        optimizerG.step()
        # Update D
        optimizerD.step()
        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (epoch % 2 == 0 and i == len(train_loader)-1):
            torch.save(gen.state_dict(), os.path.join(results_dir, "gen_{}.model".format(epoch)))
            torch.save(disc.state_dict(), os.path.join(results_dir, "disc_{}.model".format(epoch)))
            with torch.no_grad():
                fake = gen(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, pad_value=1, normalize=True, range=(-1, 1)))
            # also save normalized so we can see patterns / mode collapse
            vutils.save_image(fake,  os.path.join(results_dir, "normalized") + "/epoch" + str(epoch) + ".png", padding=2, pad_value=1, normalize=True)
            vutils.save_image(fake, os.path.join(results_dir, "unnormalized") + "/epoch" + str(epoch) + ".png", padding=2, pad_value=1, normalize=True, range=(-1, 1))

Streaming output truncated to the last 5000 lines.
[48/1000][1230/2563]	Loss_D: 1.2052	Loss_G: 0.5818	D(x): 0.9035	D(G(z)): 0.5787 / 0.5787
[48/1000][1235/2563]	Loss_D: 0.9423	Loss_G: 0.8238	D(x): 0.7745	D(G(z)): 0.4273 / 0.4273
[48/1000][1240/2563]	Loss_D: 0.9526	Loss_G: 0.7827	D(x): 0.8154	D(G(z)): 0.4492 / 0.4492
[48/1000][1245/2563]	Loss_D: 0.9722	Loss_G: 0.7369	D(x): 0.9087	D(G(z)): 0.4750 / 0.4750
[48/1000][1250/2563]	Loss_D: 0.8661	Loss_G: 0.8482	D(x): 0.8731	D(G(z)): 0.4146 / 0.4146
[48/1000][1255/2563]	Loss_D: 0.8627	Loss_G: 0.8534	D(x): 0.8798	D(G(z)): 0.4132 / 0.4132
[48/1000][1260/2563]	Loss_D: 1.0152	Loss_G: 0.7197	D(x): 0.9466	D(G(z)): 0.4880 / 0.4880
[48/1000][1265/2563]	Loss_D: 1.2562	Loss_G: 0.5805	D(x): 0.9662	D(G(z)): 0.5812 / 0.5812
[48/1000][1270/2563]	Loss_D: 1.0215	Loss_G: 0.7279	D(x): 0.7979	D(G(z)): 0.4804 / 0.4804
[48/1000][1275/2563]	Loss_D: 1.0674	Loss_G: 0.6676	D(x): 0.9268	D(G(z)): 0.5191 / 0.5191
[48/1000][1280/2563]	Loss_D: 0.9688	Loss_G: 0.8219	D(x): 0.

KeyboardInterrupt: ignored

In [ ]:
fake.min()

In [ ]:
torch.save(gen.state_dict(), os.path.join(results_dir, "gen.model"))
torch.save(disc.state_dict(), os.path.join(results_dir, "disc.model"))
if collab:
  !cp -rf '{results_dir}' '{data_dir}'

Plot training loss

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

Check how images progressed

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=2, metadata=dict(artist='Me'), bitrate=1000)
ani.save("epochs.mp4", writer=writer)